__Date:__ 02-03-2017  
__Authors:__ Noah
    
The FHNW IME has a nice Linux infrastructure. Since it makes all the necessairy environments to work with HDL and associated tools very easy we are going to use this environment instead of setting up our own.
To remotely access the systems, SSH and VNC can be used which works awesomely well!

To access the IME servers via SSH, one has to be in the FHNW networks (WLAN/LAN/VPN) and execute the following set of commands:

    $ ssh <user>@wi18as33032.adm.ds.fhnw.ch
$ <passwd>
    
with <user> and <passwd> being your FHNW user credentials.

Now a new VNC session can be started. Always _reuse_ existing sessions or close them if not needed anymore to not spam the IME servers.
Existing sessions and general options and howtos can be listed with the `vncserver` alias:

    <user>@ime_servers $ vncserver
    
The help outputs are very nice and should be self explanatory, but a new session can be opened with the command

    vncserver -new -geometry <width>x<height>
    
This command will tell you a <port> which has to be remembered. The `vncserver` command always lists ports for open sessions. In your VNC client the IME servername and port have to be entered.
As a VNC viewer TigerVNC can be used comfortably and is available and working for all OSes.
The first time you open a VNC session you will be prompted to choose a <vnc password>. This is your password when you login to VNC. Not your regular FHNW password!

    wi18as33032.adm.ds.fhnw.ch:<port>
    <vnc password>
    
Have fun with tools!

---
__Date:__ 08-03-2017  
__Authors:__ Raphael & Noah

We started working throgh the code and simulink models of the previous group.
This has worked out okay up until the point where the final filter steals 10% of the amplitude.

General tooling:
* `msh 2016_HS_P5_P6_FESP fpga` to start a shell with all tools and licenses enabled
* `sysgen` to start matlab/simulink with the Xilinx systems generator enabled
*

General filestructure:
* design_sw/Simulation/Simulink_FPGA/Filterkette/ for all the FPGA simulations and generator simulink models
    * Filterkette.m to launch sim and create necessary vars
    * Filterketten.mdl to see simulink model where signals can be seen with a scope
* design_sw/Simulation/Simulink_Filterdesign/ all the filter simulations
    * Dezimation_V2_m.m to launch general filter simulation with parts convoluted back
    
Simulation of the Filterketten works well but on the last filter a good 10% of amplitude is lost:

![](images/sim_filter_chain_amplitudes_after_each_chain.png "Filterkette Amplitudes")

Checking the frequency response of the H51 filter (last stage) we see that we lose 1dB => 0.89:

![](images/H51_filter_125MHz.png "H51 Frequency Response at 125MHz")

The above picture depicts the frequency response of the H51 filter. However, the plot is not normalized to the sampling frequency, but is instead plotted up to half the system frequency, which amounts to 62.5 MHz. Since the filter chain before the H51 filter has a decimation ratio of 125 in this example (with 625 being the global decimation ratio, and H51 being responsible for one fifth of that), we should scale the horizontal axis accordingly to a maximum value of 500 kHz (125 MHz / 125 / 2). But because we are lazy, we will instead just scale our values as needed so that we can read the correct values off the incorrectly scaled plot.

The input signal has a frequency of 1 kHz, which is 1/500th of 500 kHz. So, on the plot which is scaled to 62.5 MHz, the value corresponding to our input frequency should be at 1/500th of 62.5 MHz, which comes to 125 kHz. The below picture is the small segment of the above plot around that frequency.

![](images/H51_filter_125kHz.png "H51 Frequency Response at 125kHz")

As can be seen, the frequency response has an attenuation of 1 dB at this frequency, which is quite accurately 0.89. We therefore conclude that this is the reason why the H51 filter causes roughly 10% signal amplitude loss.

The `red_pitaya_top.sv` file resides in `design_sw/RedPitaya-master/fpga/prj/v0.94/project/redpitaya.srcs/sources_1/imports/fpga/prj/v0.94/rtl`. Completely broken. Note to self: Check out red pitaya project from scratch and implement own stuff!

After generating the bitstream everything is in the Directory `design_sw/RedPitaya-master/fpga/prj/v0.94/project/redpitaya.runs/impl_1`

`ssh root@10.84.130.54` to log onto the Red Pitaya
`rw` to mount read/write.
`scp` to `/opt/redpitaya/fpga`
`cat` of the bitstream to `/dev/xdevcfg`

---
__Date:__ 20-03-2017  
__Authors:__ Noah
    
Today I wrote the disposition for the report. It should be a good base on how to structure the report in it's core.

Furthermore I started crawling Red Pitaya docs.
It is really hard to find some good info or real docs. All there is is source code on github which is decently commented but that's all.
I would really appreciate some guide or tutorial on how to interface with the FPGA properly.

The upside of the research is that all code is open source and the devs seem to be very active on github and responsive to issues opened on said platform.

There is gereral information on the board on the [Red Pitaya Wiki](https://wiki.redpitaya.com/index.php?title=Main_Page) which seems to contain nothing but that one page. The article seems entirely outdated and lacking information.
The same goes for a [forums post](https://wiki.redpitaya.com/index.php?title=Main_Page) I found that describes partly how to interface the data path. It seems outdated.
Same goes for this [article](http://wiki.echopen.org/index.php/How_To_Program_the_FPGA_on_Red_Pitaya) that seems outdated but still contains valid information that could be of use.

Then there is some more modern [readthedocs](http://redpitaya.readthedocs.io/en/latest/) from the guys working on Red Pitaya at the moment. It seems informative but still lacks some key points.

I have opened an [issue](https://github.com/RedPitaya/RedPitaya/issues/107) on github to get more information on the state of the framework from the devs.
There appears to be a major revamp of the entire structure. The new `mercury` branch appears to be the future but is apparently still unstable.
I personally would rather work with partly unstable and fixable code than unmaintained old code that can't be fixed anyways wich seems to be the case for the master branch.

Let's hope the dev that is most busy working on the Red Pitaya project atm responds again soon and we get some more answers. Otherwise this is not gonna be fun I fear.

What's more is that we probably do not get around learning a bit of Verilog since the entire codebase is written in it. This should be doable using analogies to VHDL.

The brightest spot today was when I learned that the Red Pitaya team uses Jupyter to do testing and interfacing the Red Pitaya. Awesome!

---
__Date:__ 21-03-2017  
__Authors:__ Noah

Today I found out about an alternative SDK called [Koheron](https://www.koheron.com/blog/2016/10/03/decimator-cic-filter.html).
It seems promising and also has examples to build upon. It has to be evaluated to estimate if it is worth a shot!

I also dug into filter design with Python, just for fun. I found this [gem](http://old.myhdl.org/doku.php/projects:gcicexample) which appears to be an implementation of CIC filters in myHDL, a Python HDL framework which transpiles to VHDL/Verilog.
This article shows with numbers at the end how much cells a CIC filter uses for what order. They affirm what was suspected: CIC filters are awesome!

I also dug up the [Xilinx CIC Compiler](https://www.xilinx.com/support/documentation/ip_documentation/cic_compiler/v4_0/pg140-cic-compiler.pdf). It doesn't support filter design as far as I understand but it supports implementing t optimally tailored to the Xilinx hardware used with the ability to explicitely specify what elements on the FPGA to use if there is choice.

I also started reading on CIC filters today and they seem to be the real deal. They can decimate very effectively with just a few blocks on the FPGA. Also, Matlab seems to have some CIC filter blocks. How they are to be used is still a mystery to be solved ... Thanks usefull Matlab docs \s.